# Imports:

In [1]:
from math import pi, sin, cos
from typing import Sequence
import numpy as np
import random
import matplotlib.pyplot as plt

# Classes:

In [2]:
class Object:
            """
            the class to create and handle fractal Objects

            methods:

            shift:
                the function to apply the shift on the object
            scale:
                the function to apply the scaling on the object
            rotate:
                the function to apply the rotation on the object
            copy:
                creates a new object with the same values for attributes

            mirror:
                the function to change the type of mirroring

            getPoints:
                the function to get the points of the object

            """

            def __init__(self, startingPoint: Sequence[float], length: float, theta: float, changing: bool,
                         mirroring: float) -> None:
                """
                the function to create the fractal objects

                :param length: the length pf the object
                :type length: float
                :param theta:the rotation
                :type theta:float
                :param changing:is the object can change
                :type changing:bool
                :param mirroring:-1. if the object is mirroring,else +1.
                :type mirroring:float
                :param startingPoint:the starting point of the object stored as np.array
                :type startingPoint:Sequence[float]
                """
                self.startingPoint = np.array(startingPoint)
                self.changing = changing
                self.theta = np.array(theta)
                self.length = np.array(length)
                self.mirroring = np.array(mirroring)

            def copy(self) -> object:
                """
                the function to create a new object with the same properties as the input object

                :return:the new object
                :rtype:object
                """
                return Object(self.startingPoint.copy(),self.length.copy(),self.theta.copy(),self.changing,self.mirroring.copy())


            def scale(self, scaleParam: float) -> object:
                """
                the function to apply the scaling on the object

                :param scaleParam: the scaling parameter
                :type scaleParam: float
                :return:the exact object
                :rtype:object
                """
                self.length *= scaleParam
                return self

            def shift(self,shiftParams:np.array) -> object:
                """
                the function to apply the shift on the object
                R is the rotating matrix witch should be applied on the shift vector

                :param shiftParams:(the shift vector /length of object)
                :type shiftParams:np.array
                :return:the exact object
                :rtype:object
                """
                R = np.array([[cos(self.theta),sin(self.theta)],[-sin(self.theta),cos(self.theta)]])
                self.startingPoint += self.length*R.dot(shiftParams)
                return self

            def rotate(self, rotatingParam: float) -> object:
                """
                the function to apply the rotation on the object
                R is the rotating matrix

                :param rotatingParam: the change of theta
                :type rotatingParam: float
                :return:the exact object
                :rtype:object
                """
                self.theta += rotatingParam*self.mirroring
                return self

            def mirror(self, mirrorParam: bool) -> object:
                """
                the function to change the type of mirroring

                :param mirrorParam: the mirroring param of the transformer
                :type mirrorParam: float
                :return: the exact object
                :rtype: Object
                """
                self.mirroring *= mirrorParam
                return self
            def getPoints(self) -> Sequence[Sequence[float]]:
                """
                the function to get the points of the object

                :return:the points of the object
                :rtype:Sequence[Sequence[float]]
                """
                return [self.startingPoint[0],self.startingPoint[0]+self.length*sin(self.theta)],[self.startingPoint[1],self.startingPoint[1]+self.length*cos(self.theta)]


In [3]:
class Transformer:
    """
    the class to create and perform transformers

    methods:
    subTransform:
        to apply all changes of a transformer on an object and return aa new object
    """
    def __init__(self, scaleParam: float, rotateParam: float, shiftParams: Sequence[float], mirrorParam: float,
                 changingParam: bool) -> None:
        """
        creates transformers based on the params

        :param mirrorParam:-1. if the transformer change the mirroring of the object else +1.
        :type mirrorParam: float
        :param changingParam: if the transformer change the changing of the object
        :type changingParam: bool
        :param scaleParam:the scale coefficient of transformer
        :type scaleParam: float
        :param shiftParams:the shift vector of the transformer
        :type shiftParams: np.array
        :param rotateParam:the change of theta witch the transformer applies
        :type rotateParam: float
        """
        self.scaleParams =scaleParam
        self.rotateParam = rotateParam
        self.shiftParams = np.array(shiftParams)
        self.mirrorParam = mirrorParam
        self.changingParam = changingParam

    def subTransform(self, obj: object) -> object:
        """
        to apply all changes of a transformer on an object and return a new object
        the sort of the changes is important.
        Only if the object can change.

        :param obj:the first object
        :type obj: object
        :return: new object
        :rtype: object
        """
        newObj = obj.copy()
        if obj.changing:
            newObj.changing = self.changingParam
            return newObj.shift(self.shiftParams).scale(self.scaleParams).rotate(self.rotateParam).mirror(self.mirrorParam)
        else:
            return newObj


In [4]:
class BarnsleyFern:
    """
    the class to create Dragon highway object and simulate the fractal growth

    methods:
        visualize:
            the function to create a plotly plot of the objects at the step
        transform:
            the function to apply all the transformers on the input object
        step:
            the function to go one step forward
    """

    def getSequence(self, depth: int) -> Sequence[Transformer]:
        """
        returns a random sequence of transformers with the input size

        :param depth:the depth of fractal growth
        :type depth:int
        :return:the random list of transformers with the size of depth
        :rtype:Sequence[Transformer]
        """
        return random.choices(self.transformers,k = depth)

    def __init__(self, firstObject: object, transformers: Sequence[Transformer]) -> None:
        """
        the function to create the Dragon object with the first object and the transformers

        :param firstObject:the first input
        :type firstObject:object
        :param transformers:the transformers to apply on the objects
        :type transformers:Transformer
        """
        self.objects:np.array = [firstObject]
        self.transformers = transformers

    def visualize(self) -> None:
        """
        the function to create a plotly plot of the objects at the step
        """
        for obj in self.objects:
            plt.plot(obj.getPoints()[0],obj.getPoints()[1],color = "green")

    def transform(self, obj: object) -> Sequence[object]:
        """
        the function to apply all the transformers on the input object

        :param obj:the input object
        :type obj:object
        :return:the new output objects
        :rtype:Sequence[object]
        """
        outputs = []
        for transformer in self.transformers:
            outputs.append(transformer.subTransform(obj))

        return outputs

    def step(self) -> None:
        """
        the function to go one step forward
        """
        new_objects = []
        for obj in self.objects:
            new_objects += self.transform(obj)

        self.objects = new_objects

    def drawRandomFractal(self, depth: int, iterations: int) -> None:
        """
        draws a random Sierpinski triangle fractal plot

        :param depth:the depth of fractal growth
        :type depth:int
        :param iterations:the number of redrawing iterations
        :type iterations:int

        """

        for _ in range(iterations):
            sequence = self.getSequence(depth)
            newObj = firstObject
            for i in range(depth):
                newObj = sequence[i].subTransform(newObj)

            plt.plot(newObj.getPoints()[0],newObj.getPoints()[1],color = "green")
            del newObj


# Main Part:

### creating the first object and the transformers and the fractal object

In [5]:
firstObject = Object([0.,0.],1.,0.,True,+1.)
transformer1 = Transformer(0.9,pi/36,[0.,0.1],+1.,True)
transformer2 = Transformer(0.3,pi/4,[0.,0.05],-1.,True)
transformer3= Transformer(0.3,-pi/4,[0.,0.1],+1.,True)
transformer4 = Transformer(0.1,0,[0,0],+1.,False)
fractal = BarnsleyFern(firstObject,[transformer1,transformer2,transformer3,transformer4])

### draw the random fractal:

In [ ]:
fractal.drawRandomFractal(25,10000000)